## 6. Melhores Práticas

**✓ Boas práticas:**
- Use `PROVIDER credential_chain` sempre que possível
- Nunca faça hardcode de credenciais no código
- Use secrets persistentes para produção
- Implemente rotação periódica de credenciais
- Use scopes para limitar acesso por bucket

**✗ Evite:**
- Credenciais no código-fonte
- Secrets temporários em produção
- Mesma credencial para dev/prod

In [ ]:
import duckdb
import os
from typing import Optional, List, Dict
from dataclasses import dataclass

@dataclass
class SecretConfig:
    name: str
    type: str
    provider: Optional[str] = None
    params: Optional[Dict] = None

class SecretManager:
    """Gerenciador centralizado de secrets"""
    
    def __init__(self, db_path: str = ':memory:'):
        self.con = duckdb.connect(db_path)
        self.con.execute("INSTALL httpfs; LOAD httpfs;")
    
    def create_s3_secret(
        self,
        name: Optional[str] = None,
        use_credential_chain: bool = True,
        access_key: Optional[str] = None,
        secret_key: Optional[str] = None,
        region: str = 'us-east-1',
        scope: Optional[str] = None
    ):
        """Criar secret S3"""
        secret_clause = f"CREATE SECRET {name}" if name else "CREATE SECRET"
        
        if use_credential_chain:
            params = "TYPE S3, PROVIDER credential_chain"
        else:
            if not access_key or not secret_key:
                raise ValueError("Access key e secret key obrigatórios")
            params = f"TYPE S3, KEY_ID '{access_key}', SECRET '{secret_key}', REGION '{region}'"
        
        if scope:
            params += f", SCOPE '{scope}'"
        
        self.con.execute(f"{secret_clause} ({params})")
        print(f"✓ Secret S3 criado: {name or 'anonymous'}")
    
    def create_azure_secret(
        self,
        name: Optional[str] = None,
        account_name: Optional[str] = None,
        account_key: Optional[str] = None,
        connection_string: Optional[str] = None
    ):
        """Criar secret Azure"""
        secret_clause = f"CREATE SECRET {name}" if name else "CREATE SECRET"
        
        if connection_string:
            params = f"TYPE AZURE, CONNECTION_STRING '{connection_string}'"
        elif account_name and account_key:
            params = f"TYPE AZURE, ACCOUNT_NAME '{account_name}', ACCOUNT_KEY '{account_key}'"
        else:
            raise ValueError("Credenciais obrigatórias")
        
        self.con.execute(f"{secret_clause} ({params})")
        print(f"✓ Secret Azure criado: {name or 'anonymous'}")
    
    def list_secrets(self) -> List[Dict]:
        """Listar todos os secrets"""
        return self.con.execute("""
            SELECT name, type, provider, scope
            FROM duckdb_secrets()
        """).df().to_dict('records')
    
    def drop_secret(self, name: str):
        """Remover secret"""
        self.con.execute(f"DROP SECRET IF EXISTS {name}")
        print(f"✓ Secret removido: {name}")
    
    def test_s3_connection(self, bucket: str, path: str = '') -> bool:
        """Testar conexão S3"""
        try:
            test_path = f"s3://{bucket}/{path}" if path else f"s3://{bucket}"
            self.con.execute(f"SELECT * FROM delta_scan('{test_path}') LIMIT 1")
            print(f"✓ Conexão S3 OK: {test_path}")
            return True
        except Exception as e:
            print(f"✗ Falha na conexão: {e}")
            return False
    
    def close(self):
        self.con.close()

# Exemplo de uso
manager = SecretManager()

# Criar secrets
manager.create_s3_secret(
    name='prod_aws',
    use_credential_chain=True,
    scope='s3://production-*'
)

manager.create_s3_secret(
    name='dev_aws',
    use_credential_chain=True,
    scope='s3://development-*'
)

# Listar
print("\nSecrets configurados:")
for secret in manager.list_secrets():
    print(f"  - {secret['name']}: {secret['type']}")

manager.close()

## 5. Classe SecretManager Completa

Classe para gerenciar todos os secrets de forma centralizada.

In [ ]:
import duckdb
from datetime import datetime, timedelta

def rotate_secret_if_needed(con, secret_name, last_rotation, rotation_days=90):
    """Rotacionar secret se passou X dias"""
    days_since_rotation = (datetime.now() - last_rotation).days
    
    if days_since_rotation > rotation_days:
        # Buscar novas credenciais (de um cofre seguro)
        new_key = 'new_key_from_vault'
        new_secret = 'new_secret_from_vault'
        
        # Atualizar secret
        con.execute(f"""
            CREATE OR REPLACE SECRET {secret_name} (
                TYPE S3,
                KEY_ID '{new_key}',
                SECRET '{new_secret}'
            )
        """)
        
        print(f"✓ Secret {secret_name} rotacionado")
        return datetime.now()
    
    return last_rotation

# Exemplo de uso
con = duckdb.connect('mydb.duckdb')
last_rotation = datetime.now() - timedelta(days=100)
last_rotation = rotate_secret_if_needed(con, 'aws_prod', last_rotation)

## 4. Rotação de Credenciais

Função para rotacionar secrets periodicamente.

In [ ]:
import duckdb

con = duckdb.connect()

# Listar secrets existentes
secrets = con.execute("SELECT * FROM duckdb_secrets()").df()
print("Secrets configurados:")
print(secrets)

# Secret temporário (válido apenas na conexão)
con.execute("""
    CREATE TEMPORARY SECRET temp_s3 (
        TYPE S3,
        KEY_ID 'temp_key',
        SECRET 'temp_secret'
    )
""")

# Secret persistente (salvo no database)
con2 = duckdb.connect('mydb.duckdb')
con2.execute("""
    CREATE PERSISTENT SECRET aws_prod (
        TYPE S3,
        KEY_ID 'prod_key',
        SECRET 'prod_secret'
    )
""")

print("✓ Secrets criados")

## 3. Gerenciamento de Secrets

Listar, criar e remover secrets.

In [ ]:
import duckdb
import os

# Caminho para service account key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/to/key.json'

con = duckdb.connect()
con.execute("LOAD httpfs")

con.execute("""
    CREATE SECRET (
        TYPE GCS,
        PROVIDER credential_chain
    )
""")

print("✓ Secret GCS configurado")

## 2. Google Cloud Storage (GCS)

Autenticação com GCS usando service account.

In [ ]:
import duckdb
import os

# Configurar credenciais via environment
os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_key'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")

# Criar secret com credential chain
con.execute("""
    CREATE SECRET (
        TYPE S3,
        PROVIDER credential_chain
    )
""")

# Testar acesso
# df = con.execute("SELECT * FROM delta_scan('s3://bucket/table')").df()
print("✓ Secret S3 configurado")

## 1. Configuração Básica de Secrets S3

Autenticação com AWS S3 usando credential chain.

In [ ]:
# Instalação de dependências
%pip install duckdb deltalake pyarrow pandas -q

# Capítulo 06: Secrets e Autenticação

Gerenciamento seguro de credenciais para acessar Delta Lake em cloud storage (S3, Azure, GCS).

# Capitulo 06 Secrets Autenticacao

Notebook gerado automaticamente a partir do código fonte python.
